In [17]:
import os
import re
import numpy as np
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook

# ------------------- FUNCIONES GENERALES -------------------

def limpiar_nombre(nombre):
    return re.sub(r'[^\w\s-]', '', str(nombre)).strip().replace(' ', '_').upper()

def detectar_fuente_datos(nombre_archivo):
    nombre = nombre_archivo.lower()
    if "ccaa" in nombre:
        return "CCAA"
    elif "canales" in nombre:
        return "CANAL"
    elif "sociodem" in nombre:
        return "SOCIODEM"
    else:
        return "GENERAL"

def contiene_cabecera_reconocida(df_sample):
    # Convertir el sample a texto para búsqueda
    texto = ' '.join(df_sample.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)).lower()
    
    patrones = [
        r't\.?\s*españa',
        r'penetraci[oó]n\s*\(%\)',
        r'precio\s*medio',
        r'valor\s*\(.*euros\)',
        r'volumen\s*\(.*kg.*litros\)',
        r'consumo\s*per\s*c[aá]pita',
        r'gasto\s*per\s*c[aá]pita',
        r'canales?\s*de\s*distribuci[oó]n',
        r'socio.*econ[oó]mico'
    ]
    return any(re.search(pat, texto) for pat in patrones)

def extraer_niveles_desde_excel(path_excel):
    wb = load_workbook(path_excel, data_only=True)
    resultados = []

    for nombre_hoja in wb.sheetnames:
        ws = wb[nombre_hoja]
        niveles = []
        nombres = []

        for row in ws.iter_rows(min_row=2, max_col=1):
            cell = row[0]
            nombre = cell.value
            indent = cell.alignment.indent if cell.alignment and cell.alignment.indent else 0

            if nombre is not None:
                nombre_str = str(nombre).strip()
                nombre_upper = nombre_str.upper()

                # ✂️ FILTRAR si el texto tiene muchas repeticiones tipo "NIVEL ALTO NIVEL ALTO"
                palabras = nombre_upper.split()
                if len(palabras) >= 4:
                    palabras_unicas = set(palabras)
                    if len(palabras_unicas) <= 2 and any("NIVEL" in p for p in palabras_unicas):
                        continue  # Se descarta la fila

                nombres.append(nombre_str)
                niveles.append(indent)

        df_hoja = pd.DataFrame({"Nombre": nombres, "Nivel": niveles})
        df_hoja["Hoja"] = nombre_hoja
        resultados.append(df_hoja)

    if resultados:
        return pd.concat(resultados, ignore_index=True)
    else:
        return pd.DataFrame(columns=["Nombre", "Nivel", "Hoja"])

# ------------------- FUNCIONES PARA MENSUALES -------------------

def procesar_sociodem_por_hoja(file_path, output_folder, year, niveles_df=None):
    xls = pd.ExcelFile(file_path)
    hojas = [h for h in xls.sheet_names if h.lower() in
             ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
              'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
    if not hojas:
        print(f"  ⚠ No se encontraron hojas de meses válidas en {file_path}")
        return

    for hoja in hojas:
        print(f"  📄 Procesando hoja: {hoja}")
        df = pd.read_excel(file_path, sheet_name=hoja, header=None)
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        fila_header = 3
        cols_tespana = df.iloc[fila_header - 1].astype(str).str.upper().str.strip()
        cols_tespana_idx = cols_tespana[cols_tespana == 'T.ESPAÑA'].index.tolist()

        if len(cols_tespana_idx) < 2:
            print(f"    ⚠ No se encontraron al menos dos columnas con 'T.ESPAÑA' en hoja {hoja}")
            continue

        cols_tespana_idx.append(df.shape[1])
        primera_columna = df.iloc[fila_header - 1:, 0].reset_index(drop=True)
        nombre_primera_col = df.iloc[fila_header - 1, 0]

        column_ranges = {
            (3, 6): "CLASE SOCIAL",
            (8, 9): "NIÑOS EN EL HOGAR",
            (11, 11): "ACTIVIDAD RESPONSABLE DE COMPRA",
            (13, 15): "EDAD RESPONSABLE DE COMPRA",
            (17, 19): "TAMAÑO DE HOGAR",
            (21, 24): "TAMAÑO DEL HÁBITAT",
            (25, 25): "ÁREAS METROPOLITANAS",
            (28, 35): "TIPO DE HOGAR"
        }

        for i in range(len(cols_tespana_idx) - 1):
            start_col = cols_tespana_idx[i]
            end_col = cols_tespana_idx[i + 1]
            bloque = df.iloc[fila_header - 1:, start_col:end_col].reset_index(drop=True)
            bloque.insert(0, nombre_primera_col, primera_columna)

            fila_categoria = df.iloc[2, start_col:end_col].fillna('').astype(str).str.strip()
            fila_dato = df.iloc[3, start_col:end_col].fillna('').astype(str).str.strip()

            # 🔧 Rellenar categorías explícitamente
            for (abs_start, abs_end), base_name in column_ranges.items():
                for abs_col in range(abs_start, abs_end + 1):
                    if start_col <= abs_col < end_col:
                        rel_col = abs_col - start_col
                        fila_categoria.iloc[rel_col] = base_name

            # 🔁 Fallback automático: rellenar categorías faltantes con la última conocida
            ultima_categoria = ""
            for idx in range(len(fila_categoria)):
                if fila_categoria.iloc[idx]:
                    ultima_categoria = fila_categoria.iloc[idx]
                else:
                    fila_categoria.iloc[idx] = ultima_categoria

            # 🔧 Combinar encabezados
            columnas_combinadas = ["Alimentos"]
            for cat, dat in zip(fila_categoria, fila_dato):
                cat = str(cat).strip()
                dat = str(dat).strip()
                if cat and dat:
                    columnas_combinadas.append(f"{cat} ({dat})")
                elif dat:
                    columnas_combinadas.append(dat)
                elif cat:
                    columnas_combinadas.append(cat)
                else:
                    columnas_combinadas.append("SIN NOMBRE")

            if len(columnas_combinadas) != bloque.shape[1]:
                print(f"    ⚠ Saltando bloque {i+1} por conflicto en columnas ({len(columnas_combinadas)} != {bloque.shape[1]})")
                continue

            bloque.columns = columnas_combinadas
            bloque = bloque.drop(0).reset_index(drop=True)

            if niveles_df is not None and not niveles_df.empty:
                bloque['Alimentos'] = bloque['Alimentos'].astype(str).str.strip().str.upper()
                niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()
                niveles_df = niveles_df.drop_duplicates(subset=['Nombre'])
                bloque = bloque.merge(niveles_df, how='left', left_on='Alimentos', right_on='Nombre')
                bloque.drop(columns=['Nombre', 'Hoja'], inplace=True, errors='ignore')

            nombre_categoria = df.iloc[fila_header - 2, start_col]
            nombre_categoria = limpiar_nombre(nombre_categoria) or f"BLOQUE_{i + 1}"

            subfolder = os.path.join(output_folder, "sociodem")
            os.makedirs(subfolder, exist_ok=True)
            output_path = os.path.join(subfolder, f"{year}_sociodem_{hoja}_{nombre_categoria}.csv")
            bloque.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
            print(f"    ✔ CSV creado: {output_path}")




def extraer_datos_ccaa_nuevo(df, nombre_base, output_folder, niveles_df=None):
    for ccaa in df.columns.get_level_values(0).unique():
        sub_df = df[ccaa].copy()
        sub_df.insert(0, "Alimentos", df.index)
        sub_df = sub_df.dropna(how='all')

        if niveles_df is not None and not niveles_df.empty:
            sub_df['Alimentos'] = sub_df['Alimentos'].astype(str).str.strip().str.upper()
            niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()
            niveles_df = niveles_df.drop_duplicates(subset=['Nombre'])
            sub_df = sub_df.merge(niveles_df, how='left', left_on='Alimentos', right_on='Nombre')
            sub_df.drop(columns=['Nombre'], inplace=True)


        ccaa_limpio = re.sub(r'[\\/*?:"<>|]', "_", ccaa)
        subfolder = os.path.join(output_folder, "ccaa")
        os.makedirs(subfolder, exist_ok=True)
        output_path = os.path.join(subfolder, f"{nombre_base}_{ccaa_limpio}.csv")
        sub_df.to_csv(output_path, sep=";", encoding="utf-8-sig", index=False)
        print(f"  ✔ CCAA guardado: {output_path}")


def extraer_bloques_datos(df, nombre_base, output_folder, tipo_fuente, niveles_df=None):
    # Primero intentamos encontrar T.ESPAÑA en las primeras filas
    tespana_mask = df.apply(lambda row: row.astype(str).str.contains(r'T\.?\s*ESPAÑA', case=False, regex=True).any(), axis=1)
    tespana_rows = df[tespana_mask]
    
    # Si no encontramos T.ESPAÑA, buscamos otros patrones de cabecera
    if tespana_rows.empty:
        print(f"  ⚠ No se encontró 'T.ESPAÑA' en {nombre_base}, buscando otros patrones...")
        # Buscamos filas con patrones de tipos de datos
        tipos_dato_idx = None
        for i in range(min(20, len(df))):  # Buscar en las primeras 20 filas
            fila = df.iloc[i].dropna().astype(str).str.upper().str.strip()
            if any(fila.str.contains('PENETRACION|VALOR|VOLUMEN|PRECIO|CONSUMO\s*PER\s*C.A?PITA|GASTO\s*PER\s*C.A?PITA', case=False, regex=True)):
                tipos_dato_idx = i
                break
        
        if tipos_dato_idx is None:
            print(f"  ⚠ No se encontró fila con tipos de dato en {nombre_base}")
            return
        
        # Asumimos que la fila siguiente es la de encabezados
        encabezados = df.iloc[tipos_dato_idx].fillna('').astype(str)
        tipos_dato = df.iloc[tipos_dato_idx].fillna('').astype(str)
        encabezados_idx = tipos_dato_idx + 1
    else:
        encabezados_idx = tespana_rows.index[0]
        # Buscar fila con tipos de dato
        tipos_dato_idx = None
        for i in range(encabezados_idx-1, max(encabezados_idx-10, -1), -1):
            fila = df.iloc[i].dropna().astype(str).str.upper().str.strip()
            if any(fila.str.contains('PENETRACION|VALOR|VOLUMEN|PRECIO|CONSUMO\s*PER\s*C.A?PITA|GASTO\s*PER\s*C.A?PITA', case=False, regex=True)):
                tipos_dato_idx = i
                break
        
        if tipos_dato_idx is None:
            print(f"  ⚠ No se encontró fila con tipos de dato en {nombre_base}")
            return
        
        encabezados = df.iloc[encabezados_idx].fillna('').astype(str)
        tipos_dato = df.iloc[tipos_dato_idx].fillna('').astype(str)

    # Procesamiento de bloques (igual que antes)
    bloques = []
    current_tipo = None
    start_col = None
    for i, (encabezado, tipo) in enumerate(zip(encabezados, tipos_dato)):
        tipo = tipo.strip().upper()
        if "PENETRACION" in tipo:
            if current_tipo is not None:
                bloques.append((current_tipo, start_col, i))
            current_tipo = "PENETRACION"
            start_col = i
        elif "PRECIO MEDIO" in tipo:
            if current_tipo is not None:
                bloques.append((current_tipo, start_col, i))
            current_tipo = "PRECIO_MEDIO"
            start_col = i
        elif "VALOR" in tipo and "MILES" in tipo:
            if current_tipo is not None:
                bloques.append((current_tipo, start_col, i))
            current_tipo = "VALOR"
            start_col = i
        elif "VOLUMEN" in tipo and "MILES" in tipo:
            if current_tipo is not None:
                bloques.append((current_tipo, start_col, i))
            current_tipo = "VOLUMEN"
            start_col = i
    if current_tipo is not None:
        bloques.append((current_tipo, start_col, len(encabezados)))

    print(f"  💡 Se encontraron {len(bloques)} bloques en {nombre_base}")

    for tipo, start_col, end_col in bloques:
        tipo_limpio = re.sub(r'[^a-zA-Z0-9]', '_', tipo.lower()).strip('_')
        try:
            bloque_df = df.iloc[encabezados_idx+1:, [0] + list(range(start_col, end_col))]
            column_names = ['Alimentos'] + list(encabezados[start_col:end_col])
            if bloque_df.shape[1] != len(column_names):
                print(f"  ⚠ Saltando bloque {tipo}: número de columnas no coincide ({bloque_df.shape[1]} != {len(column_names)})")
                continue
            bloque_df.columns = column_names            
            bloque_df = bloque_df.dropna(how='all')
            bloque_df = bloque_df[bloque_df['Alimentos'].notna()]
            bloque_df = bloque_df[~bloque_df['Alimentos'].astype(str).str.contains('T\.?\s*ESPAÑA', case=False, regex=True)]
            for col in bloque_df.columns[1:]:
                bloque_df[col] = pd.to_numeric(bloque_df[col], errors='coerce')
            bloque_df = bloque_df.drop_duplicates()

            if niveles_df is not None and not niveles_df.empty:
                bloque_df['Alimentos'] = bloque_df['Alimentos'].astype(str).str.strip().str.upper()
                niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()
                niveles_df = niveles_df.drop_duplicates(subset=['Nombre'])
                bloque_df = bloque_df.merge(niveles_df, how='left', left_on='Alimentos', right_on='Nombre')
                bloque_df.drop(columns=['Nombre'], inplace=True)


            if tipo_fuente.upper() == "ANUAL":
                subfolder = output_folder  # Guarda directamente en CSV_Anuales
            else:
                subfolder = os.path.join(output_folder, tipo_fuente.lower())
            os.makedirs(subfolder, exist_ok=True)
            output_path = os.path.join(subfolder, f"{nombre_base}_{tipo.upper()}.csv")
            bloque_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
            print(f"  ✔ {tipo} guardado: {output_path}")
        except Exception as e:
            print(f"  ✖ Error procesando {tipo}: {str(e)}")

def procesar_mensuales(input_folder="Excel Mensuales", output_folder="CSV_Mensuales"):
    if not os.path.exists(input_folder):
        print(f"❌ No existe la carpeta: {input_folder}")
        return

    os.makedirs(output_folder, exist_ok=True)
    portada_keywords = ["portada", "inicio", "presentación", "resumen", "lista canales"]

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(('.xlsx', '.xls')):
            continue

        file_path = os.path.join(input_folder, filename)
        print(f"\n📂 Procesando archivo mensual: {filename}")

        match = re.search(r"(20\d{2})", filename)
        year = match.group(1) if match else "sin_anio"
        tipo_fuente = detectar_fuente_datos(filename)

        try:
            if tipo_fuente == "SOCIODEM":
                niveles_df = extraer_niveles_desde_excel(file_path)
                procesar_sociodem_por_hoja(file_path, output_folder, year, niveles_df=niveles_df)
                continue
            elif tipo_fuente in ["CCAA", "CANAL"]:
                niveles_df = extraer_niveles_desde_excel(file_path)
            else:
                niveles_df = None



            excel_data = pd.ExcelFile(file_path)


            for sheet_name in excel_data.sheet_names:
                if any(k in sheet_name.lower() for k in portada_keywords):
                    print(f"  ⏩ Omitida hoja: {sheet_name}")
                    continue

                nombre_base = f"{year}_{tipo_fuente.lower()}_{sheet_name.replace(' ', '_').lower()}"
                if tipo_fuente == "CCAA":
                    df = excel_data.parse(sheet_name, header=[1, 2], index_col=0)
                    extraer_datos_ccaa_nuevo(df, nombre_base, output_folder, niveles_df=niveles_df)
                else:
                    df = excel_data.parse(sheet_name, header=None)
                    extraer_bloques_datos(df, nombre_base, output_folder, tipo_fuente, niveles_df=niveles_df)

        except Exception as e:
            print(f"  ✖ Error procesando {filename}: {str(e)}")

    print("\n✅ Procesamiento de archivos mensuales finalizado.")

# ------------------- FUNCIONES PARA ANUALES -------------------

def procesar_anuales(input_folder="Excels Anuales", output_folder="CSV_Anuales"):
    if not os.path.exists(input_folder):
        print(f"❌ No existe la carpeta: {input_folder}")
        return
    os.makedirs(output_folder, exist_ok=True)

    portada_keywords = ["portada", "inicio", "presentación", "resumen", "lista canales"]

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(('.xlsx', '.xls')):
            continue
            
        file_path = os.path.join(input_folder, filename)
        print(f"\n📂 Procesando archivo anual: {filename}")

        match = re.search(r"(20\d{2})", filename)
        anio = match.group(1) if match else "sin_anio"
        excel_data = pd.ExcelFile(file_path)
        niveles_df = extraer_niveles_desde_excel(file_path)

        for sheet_name in excel_data.sheet_names:
            if any(k in sheet_name.lower() for k in portada_keywords):
                print(f"  ⏩ Omitida hoja: {sheet_name}")
                continue

            nombre_base = f"{anio}_anual_{sheet_name.replace(' ', '_').lower()}"

            try:
                df = excel_data.parse(sheet_name, header=None)

                # Normalizar el nombre de la hoja
                normalized_name = sheet_name.strip().lower().replace('á', 'a')

                # Detectar si contiene alguna de las palabras clave
                if 'consumoxcapita' in normalized_name or 'gastoxcapita' in normalized_name:
                    # Caso especial: solo tiene un tipo de dato
                    encabezados_idx = 0
                    while encabezados_idx < len(df) and df.iloc[encabezados_idx].isnull().all():
                        encabezados_idx += 1
                    encabezados = df.iloc[encabezados_idx].fillna('').astype(str)
                    bloque_df = df.iloc[encabezados_idx+1:].copy()
                    bloque_df.columns = encabezados
                    bloque_df = bloque_df.dropna(how='all')
                    bloque_df = bloque_df[bloque_df.iloc[:, 0].notna()]
                    bloque_df = bloque_df.drop_duplicates()

                    # RENOMBRAR LA PRIMERA COLUMNA A "Alimentos"
                    cols = list(bloque_df.columns)
                    cols[0] = "Alimentos"
                    bloque_df.columns = cols

                    # Añadir niveles si están disponibles
                    if niveles_df is not None and not niveles_df.empty:
                        bloque_df.iloc[:, 0] = bloque_df.iloc[:, 0].astype(str).str.strip().str.upper()
                        niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()
                        niveles_df = niveles_df.drop_duplicates(subset=['Nombre'])
                        bloque_df = bloque_df.merge(niveles_df, how='left', left_on=bloque_df.columns[0], right_on='Nombre')
                        bloque_df.drop(columns=['Nombre', 'Hoja'], inplace=True, errors='ignore')

                    output_path = os.path.join(output_folder, f"{nombre_base}.csv")
                    bloque_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
                    print(f"  ✔ {sheet_name.upper()} guardado: {output_path}")
                    continue  # saltar a siguiente hoja

                # Estructura estándar
                if isinstance(df.columns, pd.MultiIndex):
                    extraer_datos_ccaa_nuevo(df, nombre_base, output_folder, niveles_df=niveles_df)
                else:
                    extraer_bloques_datos(df, nombre_base, output_folder, "ANUAL", niveles_df=niveles_df)

            except Exception as e:
                print(f"  ✖ Error procesando hoja {sheet_name}: {str(e)}")
                continue

    print("\n✅ Procesamiento de archivos anuales finalizado.")


# ------------------- EJECUCIÓN -------------------

if __name__ == "__main__":

    print("=== INICIANDO PROCESAMIENTO DE MENSUALES ===")
    procesar_mensuales()

    print("\n=== INICIANDO PROCESAMIENTO DE ANUALES ===")
    procesar_anuales()



=== INICIANDO PROCESAMIENTO DE MENSUALES ===

📂 Procesando archivo mensual: 2022_datos_mensuales_panel_consumo_hogares_canales_base2021_tcm30_686041_canales.xlsx
  ⏩ Omitida hoja: PORTADA
  ⏩ Omitida hoja: Lista canales
  💡 Se encontraron 4 bloques en 2022_canal_enero
  ✔ PENETRACION guardado: CSV_Mensuales\canal\2022_canal_enero_PENETRACION.csv
  ✔ PRECIO_MEDIO guardado: CSV_Mensuales\canal\2022_canal_enero_PRECIO_MEDIO.csv
  ✔ VALOR guardado: CSV_Mensuales\canal\2022_canal_enero_VALOR.csv
  ✔ VOLUMEN guardado: CSV_Mensuales\canal\2022_canal_enero_VOLUMEN.csv


C:\Users\Jokin\AppData\Local\Temp\ipykernel_76216\1162139421.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()


  💡 Se encontraron 4 bloques en 2022_canal_febrero
  ✔ PENETRACION guardado: CSV_Mensuales\canal\2022_canal_febrero_PENETRACION.csv
  ✔ PRECIO_MEDIO guardado: CSV_Mensuales\canal\2022_canal_febrero_PRECIO_MEDIO.csv
  ✔ VALOR guardado: CSV_Mensuales\canal\2022_canal_febrero_VALOR.csv
  ✔ VOLUMEN guardado: CSV_Mensuales\canal\2022_canal_febrero_VOLUMEN.csv


C:\Users\Jokin\AppData\Local\Temp\ipykernel_76216\1162139421.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  niveles_df['Nombre'] = niveles_df['Nombre'].astype(str).str.strip().str.upper()


  💡 Se encontraron 4 bloques en 2022_canal_marzo
  ✔ PENETRACION guardado: CSV_Mensuales\canal\2022_canal_marzo_PENETRACION.csv
  ✔ PRECIO_MEDIO guardado: CSV_Mensuales\canal\2022_canal_marzo_PRECIO_MEDIO.csv
  ✔ VALOR guardado: CSV_Mensuales\canal\2022_canal_marzo_VALOR.csv
  ✔ VOLUMEN guardado: CSV_Mensuales\canal\2022_canal_marzo_VOLUMEN.csv


...

    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_CONSUMO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_GASTO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_PENETRACION.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_PRECIO_MEDIO_KG_Ó_LITROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_VALOR_MILES_EUROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_septiembre_VOLUMEN_MILES_KG_Ó_LITROS.csv
  📄 Procesando hoja: octubre


C:\Users\Jokin\AppData\Local\Temp\ipykernel_76216\1162139421.py:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_CONSUMO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_GASTO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_PENETRACION.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_PRECIO_MEDIO_KG_Ó_LITROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_VALOR_MILES_EUROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_octubre_VOLUMEN_MILES_KG_Ó_LITROS.csv
  📄 Procesando hoja: noviembre


C:\Users\Jokin\AppData\Local\Temp\ipykernel_76216\1162139421.py:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_CONSUMO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_GASTO_X_CAPITA.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_PENETRACION.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_PRECIO_MEDIO_KG_Ó_LITROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_VALOR_MILES_EUROS.csv
    ✔ CSV creado: CSV_Mensuales\sociodem\2024_sociodem_noviembre_VOLUMEN_MILES_KG_Ó_LITROS.csv

✅ Procesamiento de archivos mensuales finalizado.

=== INICIANDO PROCESAMIENTO DE ANUALES ===

📂 Procesando archivo anual: 2022-datos-anuales-panel-consumo-hogares-base2021_tcm30-686039.xlsx
  ⏩ Omitida hoja: PORTADA
  💡 Se encontraron 1 bloques en 2022_anual_valor
  ✔ VALOR guardado: CSV_Anuales\2022_anual_valor_VALOR.csv
  💡 Se encontraron 1 bloques en 2022_anual_volumen
  ✔ VOLUMEN guardado: CSV_Anuales\2022_anual_volumen_VOLUMEN.csv
  💡 Se encontraron 1 bloques en

## abajo si

In [ ]:
import pandas as pd

archivo = "Excels Anuales/2023-datos-anuales-panel-consumo-hogares-base2021_tcm30-686038.xlsx"

# Mostrar las hojas
hojas = pd.ExcelFile(archivo).sheet_names
print("Hojas disponibles:", hojas)


In [ ]:
df = pd.read_excel(archivo, sheet_name="VOLUMEN", engine="openpyxl")
print(df.head(10))


In [ ]:
print(df.columns.tolist())


In [ ]:
for val in df["CONSUMO EN HOGARES"].dropna().head(30):
    leading_chars = repr(val[:10])  # los primeros 10 caracteres
    print(f"{leading_chars} | {val}")


## REVISAR

In [ ]:
from openpyxl import load_workbook

archivo = "Excels Anuales/2023-datos-anuales-panel-consumo-hogares-base2021_tcm30-686038.xlsx"
wb = load_workbook(archivo, data_only=True)
ws = wb["VOLUMEN"]

niveles = []
nombres = []

for row in ws.iter_rows(min_row=2, max_col=1):  # asumiendo fila 1 es header, columna 1 = A
    cell = row[0]
    nombre = cell.value
    # La sangría está en cell.alignment.indent (puede ser None o un número)
    indent = cell.alignment.indent if cell.alignment and cell.alignment.indent else 0
    
    nombres.append(nombre)
    niveles.append(indent)

import pandas as pd
df_niveles = pd.DataFrame({"Nombre": nombres, "Nivel": niveles})

print(df_niveles.head(20))
